### For this project, we have decide to use a pretrained model NLLB-200 and finetune it to fit our project needs.
Model link: https://huggingface.co/facebook/nllb-200-distilled-600M

In [3]:
#Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")

tokenizer_config.json: 100%|██████████| 564/564 [00:00<?, ?B/s] 
c:\Users\duong\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\duong\.cache\huggingface\hub\models--facebook--nllb-200-distilled-600M. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
sentencepiec

In [14]:
#load the source dataset
import pandas
en_data = pandas.read_json('en-US.json', lines=True)
print(en_data.shape)
print(en_data.columns)
en_data.sample(10)

(16521, 8)
Index(['id', 'locale', 'partition', 'scenario', 'intent', 'utt', 'annot_utt',
       'worker_id'],
      dtype='object')


,id,locale,partition,scenario,intent,utt,annot_utt,worker_id
15686,16307,en-US,train,email,email_sendemail,create an email that needs to go out to my family,create an email that needs to go out to my [re...,204
16514,17173,en-US,train,email,email_query,check someone's recent emails,check someone's recent emails,10
9852,10215,en-US,train,cooking,cooking_recipe,show results for cooking lasagna from web,show results for cooking [food_type : lasagna]...,635
9172,9492,en-US,dev,play,play_music,play the current playlist,play the current playlist,568
14733,15310,en-US,train,social,social_query,can you please update me on the latest tweets ...,can you please update me on the latest [media_...,264
1482,1540,en-US,train,takeaway,takeaway_query,when will my food be ready,when will my food be ready,83
9432,9774,en-US,train,play,play_audiobook,play audiobook where we left off,play [media_type : audiobook] where we left off,597
7898,8166,en-US,train,calendar,calendar_set,please add the meeting with ms. boland on wedn...,please add the [event_name : meeting] with [pe...,314
11814,12245,en-US,test,transport,transport_query,when is the next train to glasgow from aberdeen,when is the next [transport_type : train] to [...,50
10929,11327,en-US,train,play,play_podcasts,play most recent podcast upload,play most recent podcast upload,211


In [16]:
#load target dataset
zh_data = pandas.read_json('zh-CN.json', lines=True)
print(zh_data.shape)
print(zh_data.columns)
zh_data.sample(10)

(16521, 10)
Index(['id', 'locale', 'partition', 'scenario', 'intent', 'utt', 'annot_utt',
       'worker_id', 'slot_method', 'judgments'],
      dtype='object')


,id,locale,partition,scenario,intent,utt,annot_utt,worker_id,slot_method,judgments
4509,4662,zh-CN,test,news,news_query,来自新华网的新闻,来自 [media_type : 新华网] 的新闻,38,"[{'slot': 'media_type', 'method': 'localizatio...","[{'worker_id': '10', 'intent_score': 1, 'slots..."
5614,5814,zh-CN,train,iot,iot_coffee,辣和甜大豆为主,辣和甜大豆为主,38,[],"[{'worker_id': '0', 'intent_score': 1, 'slots_..."
8050,8321,zh-CN,train,general,general_quirky,你可以告诉我更多关于那个节日的事吗,你可以告诉我更多关于那个 [event_name : 节日] 的事吗,15,"[{'slot': 'event_name', 'method': 'translation'}]","[{'worker_id': '21', 'intent_score': 1, 'slots..."
11006,11405,zh-CN,train,play,play_podcasts,请在播客选择下一集,请在播客选择 [podcast_descriptor : 下一集],15,"[{'slot': 'podcast_descriptor', 'method': 'tra...","[{'worker_id': '0', 'intent_score': 1, 'slots_..."
8107,8380,zh-CN,train,calendar,calendar_remove,取消我的日历上的任何安排,取消我的日历上的任何安排,38,[],"[{'worker_id': '10', 'intent_score': 1, 'slots..."
11372,11781,zh-CN,train,recommendation,recommendation_events,剧院正在上演什么戏剧,[place_name : 剧院] 正在上演什么戏剧,11,"[{'slot': 'place_name', 'method': 'translation'}]","[{'worker_id': '36', 'intent_score': 1, 'slots..."
5944,6158,zh-CN,dev,iot,iot_cleaning,开启吸尘器,开启 [device_type : 吸尘器],35,"[{'slot': 'device_type', 'method': 'translatio...","[{'worker_id': '29', 'intent_score': 1, 'slots..."
4400,4549,zh-CN,train,play,play_music,播放播放列表邮政服务,播放播放列表 [playlist_name : 邮政服务],11,"[{'slot': 'playlist_name', 'method': 'localiza...","[{'worker_id': '36', 'intent_score': 0, 'slots..."
15312,15923,zh-CN,train,email,email_query,检查上一个小时来自家明的邮件,检查 [time : 上一个小时] 来自 [person : 家明] 的邮件,11,"[{'slot': 'person', 'method': 'localization'},...","[{'worker_id': '10', 'intent_score': 1, 'slots..."
15116,15714,zh-CN,train,social,social_post,如果你能打给亚马逊客服我将很感激,如果你能打给 [business_name : 亚马逊客服] 我将很感激,30,"[{'slot': 'business_name', 'method': 'translat...","[{'worker_id': '26', 'intent_score': 1, 'slots..."
